In [1]:
#Import Libraires we will need
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
from pandas.io.json import json_normalize


First, let's get a list of all postal codes areas in Toronto, and their associated data including: Latitude, Longitude, Borough, and Population.

In [2]:
#Use BeautifulSoup to import neighbourhood data from the wikipedia webpage
url1 = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup1= BeautifulSoup(url1, 'lxml')


#Since there is only one table on the page, we can use "find" and get the first one
table1 = soup1.find('table')

#Get all the tables rows and put them in a list
table_rows = table1.find_all('tr')

#loop through table rows and add them to a new list
list1 = []
for tr in table_rows:
    td = tr.find_all('td')
    row = [tr.text.strip() for tr in td]
    
    #only try to add a row if the row id not blank
    if row:
        
        #if Neighborhood is Not assigned, make it the same as the Borough
        if row[2]=='Not assigned': row[2]=row[1]
        
        #Add row if Borough does not equal 'Not assigned'
        if row[1] != 'Not assigned':
            list1.append(row)

#make list into a dataframe
df_tor1 = pd.DataFrame(list1,columns=['Postal Code','Borough','Neighborhood'])

#clean data by combining rows with the same PostalCode using groupby() 
df_tor2 = df_tor1.groupby(['Postal Code', 'Borough'])['Neighborhood'].apply(','.join).reset_index()


#import csv of the Latitude and Longituge of each postal code area into a dataframe
geocodes = pd.read_csv("http://cocl.us/Geospatial_data")


#merge the original dataframe with the new dataframe
df_tor3 = pd.merge(df_tor2, geocodes, on='Postal Code', how='outer')


#import csv of the population of each postal code area into a dataframe
df_population = pd.read_csv('~\Desktop\Coursera\Applied Data Science Capstone\T1201EN.csv')

#keep only the columns Geographic Code and the population data
df_population = df_population[['Geographic code','Population, 2016']]

#rename columns
df_population.columns = ['Postal Code','Population']


#merge the main dataframe with the population dataframe
df_tor4 = pd.merge(df_tor3, df_population, on='Postal Code', how='inner')


#Show first five rows of the dataframe
df_tor4.head()



,Postal Code,Borough,Neighborhood,Latitude,Longitude,Population
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353,66108.0
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497,35626.0
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711,46943.0
3,M1G,Scarborough,Woburn,43.770992,-79.216917,29690.0
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,24383.0


Since Toronto is a very large city, let's narrow down our analysis to only one borough:  East Toronto.  

In [3]:
#create dataframe of only North York
df_et1 = df_tor4[df_tor4['Borough']=='East Toronto'].reset_index(drop=True)

#show new dataframe
df_et1

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Population
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,25044.0
1,M4K,East Toronto,"The Danforth West,Riverdale",43.679557,-79.352188,31583.0
2,M4L,East Toronto,"The Beaches West,India Bazaar",43.668999,-79.315572,32640.0
3,M4M,East Toronto,Studio District,43.659526,-79.340923,24689.0
4,M7Y,East Toronto,Business Reply Mail Processing Centre 969 Eastern,43.662744,-79.321558,10.0


Since M7Y has a population of only 10, let's drop it

In [4]:
#drop Postal Code area M7Y
df_et1 = df_et1[df_et1['Postal Code'] != 'M7Y']
df_et1

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Population
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,25044.0
1,M4K,East Toronto,"The Danforth West,Riverdale",43.679557,-79.352188,31583.0
2,M4L,East Toronto,"The Beaches West,India Bazaar",43.668999,-79.315572,32640.0
3,M4M,East Toronto,Studio District,43.659526,-79.340923,24689.0


Using the Foursquare API, we will retrieve all venues around the centre of the postal code area

In [70]:
#Foursquare info
CLIENT_ID = 'MF13OZXEYBKJ3YIM4BCQ314KOJKFLXH2DDKYFTAPLLFLRX04' # your Foursquare ID
CLIENT_SECRET = 'OM1UVINHMXAE504B0QWCVIZ1OMVB0WBPE0DBTBWI3JZ0RKXR' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version



#Code from Lab to get venues for each postal code area
def getNearbyVenues(postal_codes, names, latitudes, longitudes, population, radius=500, LIMIT=100):
    
    venues_list=[]
    for postal_code, name, lat, lng, pop in zip(postal_codes, names, latitudes, longitudes, population):
        print(postal_code, name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(postal_code,
            name, 
            lat, 
            lng, 
            int(pop),
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],
            #v['venue']['location']['postalCode'],
            v['venue']['location']['formattedAddress'][1],
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood Postal Code',
                  'Neighborhood',
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Neighborhood Population',                              
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude',
    #              'Venue Postal Code',
                  'Venue Address',
                  'Venue Category']
    
    return(nearby_venues)

In [71]:
df_et_venues = getNearbyVenues(postal_codes=df_et1['Postal Code'],
                               names=df_et1['Neighborhood'],
                               latitudes=df_et1['Latitude'],
                               longitudes=df_et1['Longitude'],
                               population=df_et1['Population'],
                               radius=5000,
                               LIMIT=44
                                  )

M4E The Beaches
M4K The Danforth West,Riverdale
M4L The Beaches West,India Bazaar
M4M Studio District


Let's take a look at our data

In [72]:
print(df_et_venues.shape)
df_et_venues.head()

(176, 10)


,Neighborhood Postal Code,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Neighborhood Population,Venue,Venue Latitude,Venue Longitude,Venue Address,Venue Category
0,M4E,The Beaches,43.676357,-79.293031,25044,The Fox Theatre,43.672801,-79.287272,Toronto ON M4E 1G2,Indie Movie Theater
1,M4E,The Beaches,43.676357,-79.293031,25044,Tori's Bakeshop,43.672114,-79.290331,Toronto ON M431E6,Vegetarian / Vegan Restaurant
2,M4E,The Beaches,43.676357,-79.293031,25044,Ed's Real Scoop,43.672630,-79.287993,Toronto ON M4E 1E9,Ice Cream Shop
3,M4E,The Beaches,43.676357,-79.293031,25044,The Beech Tree,43.680493,-79.288846,Toronto ON,Gastropub
4,M4E,The Beaches,43.676357,-79.293031,25044,Bagels On Fire,43.672864,-79.286784,Toronto ON,Bagel Shop


In [29]:
#export to csv
#export_csv = df_et_venues.to_csv (r'C:\Users\MARC-ANDRESt-Laurent\Desktop\East Toronto Venues v2.csv', index = None, header=True) 

Generate a list of all unique venue categories.  Then, keep only the restaurants.

In [82]:
df_et_venues['Venue Category'].unique

<bound method Series.unique of 0                Indie Movie Theater
1      Vegetarian / Vegan Restaurant
2                     Ice Cream Shop
3                          Gastropub
4                         Bagel Shop
5                               Park
6                             Bakery
7                              Beach
8                              Beach
9                        Coffee Shop
10                  Toy / Game Store
11                       Coffee Shop
12                         BBQ Joint
13                 Health Food Store
14                             Beach
15                    Breakfast Spot
16                    Breakfast Spot
17                 French Restaurant
18                              Café
19              Hungarian Restaurant
20                    Breakfast Spot
21                               Pub
22         Middle Eastern Restaurant
23                             Beach
24                             Beach
25                           Butcher
26     

In [73]:
#look at all venue categories and keep only the restaurants
venues_to_keep = ['American Restaurant',
'Asian Restaurant',
'Bagel Shop',
'Bakery',
'Bar',
'BBQ Joint',
'Breakfast Spot',
'Burger Joint',
'Burrito Place',
'Café',
'Cocktail Bar',
'Coffee Shop',
'Diner',
'Egyptian Restaurant',
'Ethiopian Restaurant',
'French Restaurant',
'Gastropub',
'Greek Restaurant',
'Hungarian Restaurant',
'Ice Cream Shop',
'Indian Restaurant',
'Italian Restaurant',
'Japanese Restaurant',
'Latin American Restaurant',
'Mediterranean Restaurant',
'Middle Eastern Restaurant',
'Pakistani Restaurant',
'Pizza Place',
'Pub',
'Restaurant',
'Sandwich Place',
'Snack Place',
'Taco Place',
'Tea Room',
'Thai Restaurant',
'Vegetarian / Vegan Restaurant',
'Vietnamese Restaurant',
]


df_et_venues2 = df_et_venues.loc[df_et_venues['Venue Category'].isin(venues_to_keep)]
df_et_venues2.shape

(107, 10)

Count the number of restaurants by neighbourhood

In [77]:
#determine the number of restaurants by neighbourhood
df_et2 = df_et_venues2.groupby(['Neighborhood Postal Code','Neighborhood','Neighborhood Population']).size()
df_et2=df_et2.to_frame().reset_index()
df_et2.columns=['Postal Code','Neighborhood','Population','Restaurant Count']
df_et2

,Postal Code,Neighborhood,Population,Restaurant Count
0,M4E,The Beaches,25044,24
1,M4K,"The Danforth West,Riverdale",31583,30
2,M4L,"The Beaches West,India Bazaar",32640,24
3,M4M,Studio District,24689,29


Determine the best ratio of Population to Restaurant Count by creating a new column dividing population by restaurant count.  Sort the values to find the best (highest) ratio.

In [81]:
df_et2['Pop. per Rest. Count']=df_et2['Population']/df_et2['Restaurant Count']
df_et2.sort_values(by='Pop. per Rest. Count',ascending=False, inplace=True)
df_et2

,Postal Code,Neighborhood,Population,Restaurant Count,Pop. per Rest. Count
2,M4L,"The Beaches West,India Bazaar",32640,24,1360.000000
1,M4K,"The Danforth West,Riverdale",31583,30,1052.766667
0,M4E,The Beaches,25044,24,1043.500000
3,M4M,Studio District,24689,29,851.344828


We have found our answer!! The analysis shows that the postal code area M4L has the best Population to Restaurant ratio (1360:1).  So, the best postal code area to open a new restaurant in East Toronto is M4L.